# Recurrent Neural Networks
In this exercise, we will implement a simple one-layer recurrent neural network. We will use the formula for an [Elman RNN](https://en.wikipedia.org/wiki/Recurrent_neural_network#Elman_networks_and_Jordan_networks), one of the most basic and classical RNNs. The hidden state update and output at time $t$ are defined like this:

$$
\begin{align}
h_t &= \tanh(W_xh x_t + W_hh h_{t-1} + b_h) \\
y_t &= \tanh(W_hy h_t + b_y)
\end{align}
$$

In [1]:
import torch
import torch.nn as nn

We start by defining the RNN as a subclass of `nn.Module`. The network's parameters are created in the `__init__` method. Use `input_dim`, `hidden_dim` and `output_dim` as arguments that define the dimensionality of the input/hidden/output vectors. Define your parameters as `nn.Parameter` with the appropriate dimensions. The documentation of `torch.nn` can be found [here](https://pytorch.org/docs/stable/nn.html).

In [2]:
class RNN(nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim):
    super().__init__()

    self.input_dim = input_dim
    self.hidden_dim = hidden_dim
    self.output_dim = output_dim

    self.Wxh = nn.Parameter(torch.zeros(hidden_dim, input_dim))
    self.bh = nn.Parameter(torch.zeros(hidden_dim))

    self.Whh = nn.Parameter(torch.zeros(hidden_dim, hidden_dim))

    self.Why = nn.Parameter(torch.zeros(output_dim, hidden_dim))
    self.by = nn.Parameter(torch.zeros(output_dim))

Add a function `reset_parameters` that initializes your parameters. Pick a suitable distribution from [nn.init](https://pytorch.org/docs/stable/nn.init.html).

In [3]:
def reset_parameters(self):
  for weight in self.parameters():
    nn.init.uniform_(weight, -1, 1)

RNN.reset_parameters = reset_parameters

In [6]:
model = RNN(5, 20, 10)
model.reset_parameters()

print(model.Wxh)

Parameter containing:
tensor([[-0.8278, -0.5547, -0.1921,  0.6612,  0.2273],
        [ 0.5546, -0.0260,  0.6764,  0.1241,  0.5879],
        [-0.3801,  0.5282,  0.0201,  0.1527, -0.5861],
        [ 0.9658,  0.6877,  0.7096, -0.3710,  0.3886],
        [ 0.6928, -0.0716, -0.3360, -0.1884, -0.5552],
        [ 0.0991,  0.0216,  0.9750, -0.5092, -0.1958],
        [ 0.4404,  0.5275,  0.9211,  0.5052, -0.6219],
        [ 0.9136,  0.3746, -0.0532,  0.3347,  0.5139],
        [ 0.2334, -0.2251, -0.1946,  0.6618, -0.1815],
        [-0.4530, -0.2071,  0.2541, -0.3014,  0.5522],
        [ 0.7243, -0.7846,  0.7114, -0.1810,  0.9333],
        [ 0.3587,  0.6180,  0.7776,  0.4916, -0.6799],
        [-0.7767,  0.8537,  0.1152, -0.7410, -0.0965],
        [-0.4894,  0.8631,  0.2323, -0.3421, -0.7945],
        [-0.3131, -0.9024,  0.3867,  0.0524, -0.2219],
        [-0.5466,  0.9316,  0.5868, -0.7733,  0.1568],
        [-0.8027, -0.4349, -0.1382,  0.3774, -0.1256],
        [ 0.0493, -0.4355, -0.4080,  0.0461

Add a `forward` function that takes an input and a starting hidden state $h_{t-1}$ and returns the updated hidden state $h_t$ and output $y$ as outputs. The initial hidden state $h_0$ can be initialized randomly/to all zeros.

$$
\begin{align}
h_t &= \tanh(W_xh x_t + W_hh h_{t-1} + b_h) \\
y_t &= \tanh(W_hy h_t + b_y)
\end{align}
$$

In [7]:
def forward(self, x, hidden_state):
  h = torch.tanh(self.Wxh @ x + self.Whh @ hidden_state + self.bh)
  y = torch.tanh(self.Why @ h + self.by)
  return y, h

RNN.forward = forward

Test your RNN with a single input.

In [ ]:
x = torch.randn(model.input_dim)
h0 = torch.zeros(model.hidden_dim)

y, h1 = model(x, h0)
print(f"y shape: {y.shape}, y: {y}")
print(f"h1 shape: {h1.shape}, h1: {h1}")

y shape: torch.Size([10]), y: tensor([-0.2169,  0.2612, -0.3582,  0.8962, -0.9923,  0.8160, -0.9656, -0.5838,
         0.6149, -1.0000], grad_fn=<TanhBackward0>)
h1 shape: torch.Size([20]), h1: tensor([ 0.0754, -0.5543, -0.3057, -0.7539,  0.9852, -0.9889, -0.8724,  0.7625,
         0.9101, -0.6873, -0.7151, -0.9135, -0.9540, -0.9864, -0.7816, -0.9956,
        -0.8441,  0.6893,  0.3616,  0.6097], grad_fn=<TanhBackward0>)


Now create an input sequence and run it through your RNN.

In [11]:
seq_length = 4
inputs = [torch.randn(model.input_dim) for _ in range(seq_length)]
hidden_state = torch.zeros(model.hidden_dim)
outputs = []

for x in inputs:
  y, new_hidden_state = model(x, hidden_state)
  hidden_state = new_hidden_state
  outputs.append(y)

print(f"Final output: {y}")
print(f"Final hidden state: {hidden_state}")

Final output: tensor([-0.0521, -0.9989, -1.0000, -0.9450, -0.3923,  0.9226, -0.5971, -0.8471,
         0.3136, -0.8018], grad_fn=<TanhBackward0>)
Final hidden state: tensor([ 0.5730, -0.8052, -0.1531,  0.9552,  0.9985, -0.9901,  0.9998, -0.5803,
         0.0344,  0.4998, -0.9998, -0.9782,  0.9985, -0.7673,  0.9984, -0.5978,
        -0.9998, -0.4471,  0.6496, -0.8426], grad_fn=<TanhBackward0>)


The final hidden state encodes all the information present in the input sequence. It can be used as a feature for classification, or to initialize a decoder RNN to do translation, for example.

Now look at PyTorch's documentation for the [`nn.RNN`](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html) and the [`nn.RNNCell`](https://pytorch.org/docs/stable/generated/torch.nn.RNNCell.html) classes. What is the difference between the two? What is the difference to the definition from Wikipedia we used above? Run your input sequence through both the `nn.RNN` and the `nn.RNNCell`.